# Pràctica MapReduce

## 1. Instal·lar MapReduce

In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

--2022-02-15 23:07:05--  https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500749234 (478M) [application/x-gzip]
Saving to: ‘hadoop-3.3.0.tar.gz’

hadoop-3.3.0.tar.gz 100%[===================>] 477.55M  25.9MB/s    in 19s     

2022-02-15 23:07:24 (24.9 MB/s) - ‘hadoop-3.3.0.tar.gz’ saved [500749234/500749234]



In [ ]:
# Descomprimir
!tar -xzf hadoop-3.3.0.tar.gz

In [ ]:
# Copiar a /usr/local/
!cp -r hadoop-3.3.0/ /usr/local/

## 2: Configurar el Hadoop JAVA HOME

In [ ]:
# Cercam la direcció de Java en la màquina Google Colab
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [ ]:
# Establim mitjançant codi Python el valor a la variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"

## 3. Executar Hadoop

In [ ]:
!/usr/local/hadoop-3.3.0/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

## Primera part:  Carregar les dades al hdfs

### Cloudera

**script.sh**

creem i obrim un script sh
```
touch script.sh
nano script.sh
```

- Cream una carpeta i ens ubiquem a ella
- Instal·lem 5 llibres de  www.gutemberg.org
- Carreguem els llibres al hdfs

```
#! /bin/bash

sudo mkdir books
cd books

wget https://www.gutenberg.org/files/57654/57654-0.txt
wget https://www.gutenberg.org/cache/epub/22045/pg22045.txt
wget https://www.gutenberg.org/files/61339/61339-0.txt
wget https://www.gutenberg.org/cache/epub/29640/pg29640.txt
wget https://www.gutenberg.org/cache/epub/54430/pg54430.txt

hdfs dfs -put /home/cloudera/books* /user/cloudera
hdfs dfs -ls books

```

- executariem l'script amb 
```
./script.sh
```



### Colab

In [ ]:
# Creem una carpeta
!mkdir books

In [ ]:
cd books

/content/books


In [ ]:
# Descarreguem 5 llibres
!wget https://www.gutenberg.org/files/57654/57654-0.txt
!wget https://www.gutenberg.org/cache/epub/22045/pg22045.txt
!wget https://www.gutenberg.org/files/61339/61339-0.txt
!wget https://www.gutenberg.org/cache/epub/29640/pg29640.txt
!wget https://www.gutenberg.org/cache/epub/54430/pg54430.txt

--2022-02-15 23:08:49--  https://www.gutenberg.org/files/57654/57654-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1131399 (1.1M) [text/plain]
Saving to: ‘57654-0.txt’

57654-0.txt         100%[===================>]   1.08M  --.-KB/s    in 0.1s    

2022-02-15 23:08:49 (8.99 MB/s) - ‘57654-0.txt’ saved [1131399/1131399]

--2022-02-15 23:08:50--  https://www.gutenberg.org/cache/epub/22045/pg22045.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149982 (146K) [text/plain]
Saving to: ‘pg22045.txt’

pg22045.txt         100%[===================>] 146.47K  --.-KB/s    in 0.08s   

2022-02-15 23:08:50 (1.76 M

In [ ]:
# Comprovem que s'han descarregat
!ls

57654-0.txt  61339-0.txt  pg22045.txt  pg29640.txt  pg54430.txt


In [ ]:
cd ..

/content


## Segona part. Programa map reduce

Per realitzar aquesta part m'he ajudat dels Colabs del Moodle i d'alguns exemples trobats a internet com:

- https://www.geeksforgeeks.org/hadoop-streaming-using-python-word-count-problem/

**mapper.py**

Llegirà les dades de STDIN i dividirà les línies en paraules, generarà una sortida de la primera lletra de cada paraula

```
# importam sys per llegir i escriure a STDIN i STDOUT
from nltk.corpus import stopwords
import sys
import io
import re
import nltk
nltk.download('stopwords', quiet=True)

# signes que volem evitar
signs = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

# Creem les stopwords dels idiomes que tenim en els llibres
stop_words = set(stopwords.words('catalan') + stopwords.words('spanish'))
input_stream = io.TextIOWrapper(sys.stdin.buffer, encoding='utf-8')

for line in input_stream:
    # eliminem els espais en blanc del principi i final
    line = line.strip()
    line = re.sub(r'[^\w\s]', '', line)
    # convertim tot en minúscules
    line = line.lower()
    # sustituim els signes per espais
    for char in line:
        if char in signs:
            line = line.replace(char, " ")

    # De cada paraula comprovem que no sigui una stopword
    words = line.split()
    for word in words:
        if word not in stop_words:
            # primera lletra de la paraula
            letter = word[0:1]
            # si la lletra es troba a la llista farem el seu compteig
            if letter in list('abcdefghijklmnñopqrstuvwxyzç'):
                print('%s\t%s' % (letter, 1))
```

**reducer.py**

Implementa la lògica de reducció. Llegirà la sortida de mapper.py des de STDIN i afegirà l'aparició de cada lletra i escriurà la sortida final a STDOUT

```
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    line = line.lower()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    try:
        count = int(count)
    except ValueError:
        # count was not a number, ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print('%s\t%s' % (current_word, current_count))
        current_count = count
        current_word = word

# output the last word
if current_word == word:
    print('%s\t%s' % (current_word, current_count))

```

### Tercera part: Executar l’aplicació MapReduce 

In [ ]:
# Descarregam el mapper.py del github
!wget https://raw.githubusercontent.com/InesArrom/mapreduce/main/mapper.py

--2022-02-15 23:09:32--  https://raw.githubusercontent.com/InesArrom/mapreduce/main/mapper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243 (1.2K) [text/plain]
Saving to: ‘mapper.py’

mapper.py           100%[===================>]   1.21K  --.-KB/s    in 0s      

2022-02-15 23:09:32 (70.5 MB/s) - ‘mapper.py’ saved [1243/1243]



In [ ]:
# Descarregam el reducer.py del github
!wget https://raw.githubusercontent.com/InesArrom/mapreduce/main/reducer.py

--2022-02-15 23:09:35--  https://raw.githubusercontent.com/InesArrom/mapreduce/main/reducer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871 [text/plain]
Saving to: ‘reducer.py’

reducer.py          100%[===================>]     871  --.-KB/s    in 0s      

2022-02-15 23:09:35 (46.7 MB/s) - ‘reducer.py’ saved [871/871]



In [ ]:
# Cercem la ruta per executar map reduce
!find / -name 'hadoop-streaming*.jar'

/usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
/usr/local/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-sources.jar
/usr/local/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-test-sources.jar
find: ‘/proc/38/task/38/net’: Invalid argument
find: ‘/proc/38/net’: Invalid argument
/content/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
/content/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-sources.jar
/content/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-test-sources.jar


In [ ]:
# Executem el mapreduce
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/books -output /content/output -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

2022-02-15 23:10:01,247 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper.py, /content/reducer.py] [] /tmp/streamjob15741950919872639929.jar tmpDir=null
2022-02-15 23:10:02,316 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-15 23:10:02,599 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-15 23:10:02,599 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-02-15 23:10:02,622 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-02-15 23:10:02,838 INFO mapred.FileInputFormat: Total input files to process : 5
2022-02-15 23:10:02,863 INFO mapreduce.JobSubmitter: number of splits:5
2022-02-15 23:10:03,366 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1883455017_0001
2022-02-15 23:10:03,366 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-02-15 23:10:03,862 INFO mapred.Lo

In [ ]:
# comprovam al directori output si ens ha creat un arxiu _SUCCESS que ha anat be i l'arixu amb els resultats del mapreduce
!ls /content/output

part-00000  _SUCCESS


In [ ]:
# visualitzem el resultat
!cat /content/output/part-00000

a	25373
b	5695
c	25657
d	21237
e	12864
f	7869
g	5705
h	11195
i	7731
j	3789
k	49
l	8646
m	15811
n	4893
o	6257
p	23641
q	2850
r	9381
s	16390
t	14253
u	1865
v	12792
w	1054
x	1486
y	633
z	125
ñ	4
